# Funtionalities


read_geometries


Input: .Json file.
        From Shape with Geopandas (gpd). Return shapely.geometry.polygon.Polygon.
        buffer size: For improve the point representation in a raster file need to assing a buffer area for point geometry
        From Json with Geopandas (gpd). 
Return shapely.geometry.polygon.Polygon.

composite

Input: Image_path, band dataset from .SAFE (10_m) Path.
       Out_path, path to save the return.
       rgb=[4,3,2] bands to composite the RGB. The user can do another composite. exameple: False_color [8, 4, 3]
       https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/composites/
       

vector_to_raster

Input
From vector (shapely.geometry.polygon.Polygon) to raster.  
Return an image array with input geometries burned in.

https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html
https://rasterio.readthedocs.io/en/latest/topics/features.html


mask_raster

Create a raster mask from vectorial geometries.

Input:  .tiff file.
        shapes, vector information in an Iterable object (shapely.geometry.polygon.Polygon).
        out_path, path to save the return.
        Invert, If False (default) pixels outside shapes will be masked. If True, pixels inside shape will be masked

Return: mask in np.array 


get_coords

Get the coordenates of the geometries (polygons or points) from a raster file (mask)

Inputs: .tiff file 
        out_path, path to save the return in shape format
        
Return: Geodatabase with geometries coordenates 
        

mask_and_crop_raster

This function is same that mask_raster function but the shapes is a list instead of an iterator that allows crop the raster in the each polygons.       

In [79]:
import itertools
import string
import random


import earthpy.spatial as es
import earthpy.plot as ep
import geopandas as gpd
import glob
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import rasterio as rio


from pathlib import Path
from PIL import Image
from numpy import ma
from rasterio.enums import Resampling
from rasterio.mask import mask
from rasterio.plot import plotting_extent
from shapely.geometry import Point, mapping, shape
from shapely import wkt
from shutil import copyfile

In [80]:
def read_geometries(input_, in_crs, out_crs, buffer = None, file=False):


    if file:
        file = Path(input_)
        # This is a json file
        if file.suffix == '.json':
            gdf = gpd.read_file(file).to_crs(out_crs) #project the coordenate system to raster CRS
            geometries = [row['geometry'] for i, row in gdf.iterrows()] #read all geometries in the file
        elif file.suffix == '.shp':
            geometries = [row.geometry for idx, row in gpd.read_file(file).iterrows()]
        else:
            print(f'This {file.name} is not supported.')
            return
    
    elif isinstance(input_, str):
        # This is a string
        # We assume this is a WKT
        wkts = [wkt.loads(input_)]
        gdf = gpd.GeoDataFrame(geometry=wkts)
        gdf = gdf.set_crs(in_crs).to_crs(out_crs)
        geometries = [row['geometry'] for i, row in gdf.iterrows()]
    
    else:
        print(f'{input_} no supported')
        return
    
    shapes = []
    
    if buffer:
        shapes = [g.buffer(buffer).envelope for g in geometries if g.type=='Point'] #buffer to points geometries (improve the visibility)
    
    shapes.extend([g for g in geometries if g.type=='Polygon']) #join all the geometries
    
    return shapes
    

In [81]:
# def open_file(file, buffer = 10):
#     file = Path(file)
#     gdf = gpd.read_file(file).to_crs('EPSG:32631') #project the coordenate system to raster CRS
#     geometries = [row['geometry'] for i, row in gdf.iterrows()] #read all geometries in the file
#     shapes = [g.buffer(buffer).envelope for g in geometries if g.type=='Point'] #buffer to points geometries (improve the visibility)
#     shapes.extend([g for g in geometries if g.type=='Polygon']) #join all the geometries
    
#     return shapes
    

In [82]:
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)

In [83]:
def plot_composition(raster):
    src = rio.open(raster, mode='r')
    true_color = src.read()
    b1 = normalize(true_color[0])
    b2 = normalize(true_color[1])
    b3 = normalize(true_color[2])
    nrg = np.dstack((b1, b2,b3))
    return plt.imshow(nrg)

In [84]:
def get_bands_dict(image_path):
    
    band_path = glob.glob(os.path.join(image_path, '*_B*.jp2'))
    bands = {int(a[a.find('_B')+2:a.find('_B')+4]):a for a in band_path}
    
    return bands

In [85]:
def composite(image_path, out_path, rgb=[4,3,2]):
    
    bands = get_bands_dict(image_path)
    
    if len(list(set(bands.keys()) & set(rgb))) != 3:
        print(f'One or more input rgb({rgb}) bands not exists in source path')
        return
    
    b = rio.open(bands[rgb[2]], driver='JP2OpenJPEG') #blue
    g = rio.open(bands[rgb[1]], driver='JP2OpenJPEG') #green
    r = rio.open(bands[rgb[0]], driver='JP2OpenJPEG') #red
        
    #Empty raster write the composite
    emp = rio.open(out_path,'w',driver='Gtiff',
                        width=r.width, 
                        height=r.height, #width and height of any band
                        count=3,
                        crs=r.crs, #coordenate system
                        transform=r.transform, #Transfor from pixel coordinates of source to csr of the input shapes
                        dtype=r.dtypes[0])

    #combine the bands RGB in empty raster
    print('Making composite...')
    emp.write(b.read(1),3) #blue
    emp.write(g.read(1),2) #green
    emp.write(r.read(1),1) #red
    emp.close()
    
    plot_composition(out_path)
    

    
    print(f'Done in {out_path}')

In [86]:
def vector_to_raster(raster, shapes):
    raster =  rio.open(raster)
    geometries_raster = rio.features.rasterize(((g,255) for g in shapes), out_shape=raster.shape, crs=raster.crs, transform= raster.transform, all_touched=False)
    display(Image.fromarray(geometries_raster))
    
    return geometries_raster

In [87]:
def mask_all_raster(raster, shapes, out_path, invert=True):
    
    raster = rio.open(raster)
    masked = rio.mask.mask(raster, shapes, all_touched=True, invert=invert)

    with rio.Env():

        profile = raster.profile #to get the profile
        profile.update(compress='lzw')


        with rio.open(out_path, 'w', **profile) as dst:
                dst.write(masked[0], 1)
                

        msk = rio.open(out_path)        
        plt.figure(figsize=(8,10))
        plot.show(msk, transform=msk.transform)
        plt.imshow(msk[0].read(1), cmap='pink')
        
        return display(plt.imshow(msk.read(1), cmap='pink'))

In [88]:
def get_coords(raster, outpath): #.tif outpath in .shp
    with rio.open(raster) as src:
        profile = src.profile
        file = src.read(1).astype(np.float32)
        coordenates = []
        coordenate_list = []
        for shp, val in rio.features.shapes(file, transform=profile['transform']):
            coordenates.append(shape(shp))
            coordenate_list.append(shp)

    d = {'geometry': coordenates}
    gdf = gpd.GeoDataFrame(d, index=range(len(coordenates)))
    gdf.to_file(driver='ESRI Shapefile', filename=outpath)  
    
    
    return gdf, coordenate_list

In [89]:
def read_bands(image_path, levels=1, suffix='.jp2'):
    level="/".join('*'*levels)
    band_path = glob.glob(os.path.join(image_path, level ,f'*_B*{suffix}'))
    bands = {(a[a.find('_B')+2:a.find('_B')+8]):a for a in band_path}

    return bands

In [90]:
def copy_bands(inpath, outpath):
    """Copy/move bands with 10m resolution
    """
    name = Path(inpath).name
    
    if name.find('10m') != -1:
        dst_path = os.path.join(outpath, name)
        if not os.path.exists(dst_path) : copyfile(inpath, dst_path)
    

In [91]:
def upscale(inpath, outpath, reference_image):

    current_bands = [k[:3] for k in read_bands(outpath, levels=0, suffix='.tif').keys()]
    if inpath[inpath.find('_B')+2:inpath.find('_B')+5] in current_bands:
        return

    band = rio.open(inpath)
    res = band.res[0]
    
    inpath = Path(inpath)
    name = inpath.name[:-4]
    name = name.replace(str(int(res))+'m','10m')
    out_name = f'{name}_resample.tif'
    
    if res == 10:
        return
    
    upscale_factor = 2 if res == 20 else 6 
    
    
    print(f'Upscaling {name}...')
          
    reference_band = rio.open(reference_image)
    profile = reference_band.profile #to get the profile
    
    data = band.read(
        out_shape=(
            band.count,
            int(band.height * upscale_factor),
            int(band.width * upscale_factor)
        ),
        resampling=Resampling.bilinear)
    
    transform = band.transform * band.transform.scale(
                                        (band.width / data.shape[-1]),
                                        (band.height / data.shape[-2])
                                )
    profile.update(transform=transform, 
                   driver='Gtiff')
    
    with rio.open(os.path.join(outpath, out_name), 'w', **profile) as dst:
        dst.write(data[0], 1)
    print(f'Done {out_name}')
        

In [92]:
def instantiate_bands(bands_dict, dtype=np.int16):
    """
    Args:
        bands_dict (dict): Read from read_bands dictionary
    """ 
    
    d ={}
    for key in bands_dict.keys():        
        k = 'b'+ key.replace('_10m', '')
        opened = rio.open(bands_dict[key])
        
        d[k] = opened.read().astype(dtype)
        
    return d, opened
   

In [93]:
def NDWI_mask (band_dic, band_reference, outpath):
    
    with np.errstate(divide='ignore', invalid='ignore'):
        NDWI = ((band_dic['b03']-band_dic['b08'])/(band_dic['b03']+band_dic['b08']))
        NDWI_class = np.where(NDWI < 0, 1, np.nan)
    

    profile = band_reference.profile
    profile.update(driver='Gtiff',
               dtype = 'float64', 
               compress='lzw')
    
    with rio.open((outpath), 'w', **profile) as dst:
        dst.write(NDWI_class[0], 1)
        
    return NDWI_class

In [94]:
# def apply_estimator_mask(estimator):
        
#         print(estimator)
#         empty = np.zeros(estimator.shape).astype(np.float64)
        
#         if estimator < -1:
#             return empty+0
#         elif estimator > 1:
#             return empty+1
#         else:
#             return empty + ((1 + estimator) / 2)

In [95]:
def run_estimator(band_dic, band_reference, outpath):
    
#     a = np.multiply(10000, 
#                 (-0.0003402*band_dic['b01'] \
#                 -(0.0004585*band_dic['b02']) \
#                 + (0.001415*band_dic['b03']) \
#                 + (0.01254*band_dic['b04']) \
#                 - (0.01112*band_dic['b05']) \
#                 - (0.01346*band_dic['b06']) \
#                 + (0.002762*band_dic['b07']) \
#                 + (0.002481*band_dic['b08']) \
#                 + (0.009605*band_dic['b8A']) \
#                 + (0.001247*band_dic['b09']) \
#                 - (0.01462*band_dic['b11']) \
#                 + (0.00406*band_dic['b12']))
#                )
    
#     estimator = -1.76e-05+a

    estimator = -1.76e-05 + 10000*(-0.0003402*band_dic['b01'] -0.0004585*band_dic['b02'] + 0.001415*band_dic['b03'] + 0.01254*band_dic['b04'] -0.01112*band_dic['b05'] -0.01346*band_dic['b06'] + 0.002762*band_dic['b07'] + 0.002481*band_dic['b08'] + 0.009605*band_dic['b8A'] + 0.001247*band_dic['b09'] -0.01462*band_dic['b11'] + 0.00406*band_dic['b12'])

    estimator_mask = np.where(estimator <-1, 0,
                         np.where(estimator >1, 1,(1+estimator)/2))
                              
    profile = band_reference.profile
    profile.update(driver='Gtiff',
               dtype = 'float64', 
               compress='lzw')
    
    with rio.open(outpath, 'w', **profile) as dst:
        dst.write(estimator_mask[0], 1)

    return estimator_mask

In [96]:
def represent_save_mask_raster (b1, b2, b3, reference):

    b1 = normalize(result[0])
    b2 = normalize(result[1])
    b3 = normalize(result[2])
    nrg = np.dstack((b1, b2,b3))
    plt.imshow(nrg)


    emp = rio.open('new_RGB_mask.tif','w',driver='Gtiff',
                            width=reference.width, 
                            height=reference.height, #width and height of any band
                            count=3,
                            crs=reference.crs, #coordenate system
                            transform=reference.transform, #Transfor from pixel coordinates of source to csr of the input shapes
                            dtype=reference.dtypes[0])

    #combine the bands RGB in empty raster   
    emp.write(result[0],3) #blue
    emp.write(result[1],2) #green
    emp.write(result[2],1) #red
    emp.close()

In [97]:
def cividis (x):
    #  x must be in [0,1]
    #  https://github.com/matplotlib/matplotlib/blob/master/lib/matplotlib/_cm_listed.py
    return np.array([x*0.995737, x*0.909344+(1-x)*0.135112, x*0.217772+(1-x)*0.304751]).astype(np.float64)

In [98]:
def clip_raster_with_shape(raster_path, shapes, suffix='clip'):
    
    out_folder = Path('./clipped_bands').mkdir(parents=True, exist_ok=True)
    
    raster_to_clip = rio.open(raster_path)
    
    out_image,out_transform = rio.mask.mask(raster_to_clip,
                                        shapes,
                                        crop=True)
    
    clip_profile = raster_to_clip.profile
    clip_profile.update(height=out_image.shape[-2], 
                        width=out_image.shape[-1],
                        driver='Gtiff',
                        transform=out_transform)
    
    
    img_path = Path(raster_to_clip.name)
    out_clip_r_name = f'{img_path.name[:-4]}_{suffix}.tif'
    
    
    
    with rio.open(os.path.join('./clipped_bands', out_clip_r_name), 'w', **clip_profile) as dst:
        dst.write(out_image[0], 1)
        
    print(f'...Saving in {out_clip_r_name}')

# 1. Procedure

## 1.1. Extract files

In [99]:
# import zipfile
# with zipfile.ZipFile("S2A_MSIL2A_20201018T153621_N0214_R068_T18PVS_20201018T193619.zip","r") as zip_ref:
#     zip_ref.extractall()

## 2. Read bands

In [100]:
inpath = r'./S2A_MSIL2A_20201018T153621_N0214_R068_T18PVS_20201018T193619.SAFE/GRANULE/L2A_T18PVS_A027807_20201018T153841/IMG_DATA/'
bands = read_bands(inpath, levels=1, suffix='.jp2')

In [101]:
bands

{'03_10m': './S2A_MSIL2A_20201018T153621_N0214_R068_T18PVS_20201018T193619.SAFE/GRANULE/L2A_T18PVS_A027807_20201018T153841/IMG_DATA/R10m/T18PVS_20201018T153621_B03_10m.jp2',
 '08_10m': './S2A_MSIL2A_20201018T153621_N0214_R068_T18PVS_20201018T193619.SAFE/GRANULE/L2A_T18PVS_A027807_20201018T153841/IMG_DATA/R10m/T18PVS_20201018T153621_B08_10m.jp2',
 '04_10m': './S2A_MSIL2A_20201018T153621_N0214_R068_T18PVS_20201018T193619.SAFE/GRANULE/L2A_T18PVS_A027807_20201018T153841/IMG_DATA/R10m/T18PVS_20201018T153621_B04_10m.jp2',
 '02_10m': './S2A_MSIL2A_20201018T153621_N0214_R068_T18PVS_20201018T193619.SAFE/GRANULE/L2A_T18PVS_A027807_20201018T153841/IMG_DATA/R10m/T18PVS_20201018T153621_B02_10m.jp2',
 '8A_20m': './S2A_MSIL2A_20201018T153621_N0214_R068_T18PVS_20201018T193619.SAFE/GRANULE/L2A_T18PVS_A027807_20201018T153841/IMG_DATA/R20m/T18PVS_20201018T153621_B8A_20m.jp2',
 '12_20m': './S2A_MSIL2A_20201018T153621_N0214_R068_T18PVS_20201018T193619.SAFE/GRANULE/L2A_T18PVS_A027807_20201018T153841/IMG_DAT

## 3. Reescale bands to 10 m

In [102]:
%%time
outpath = './resampled_bands'
reference_image = bands['02_10m']
# Apply upscale function to all bands with >10m resolution
rescaled_arrays = [upscale(bands[key], outpath,  reference_image) 
                   if key[:3] not in [k[:3] for k in bands.keys() if '10m' in k] 
                   else copy_bands(bands[key], outpath)
                   for key in bands.keys()
                  ] 

#No duplicated bands...if already have 10 meters resolution, dont resampled this chanel 
print('Done!!!')

Done!!!
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.76 ms


## 4. Read-store reescaled bands
### 4.1. Read reescaled bands and store their path in a bands dictionary

In [103]:
bands_dpath = read_bands(outpath, levels=0, suffix='.*')

In [104]:
bands_dpath

{'8A_10m': './resampled_bands/T18PVS_20201018T153621_B8A_10m_resample.tif',
 '12_10m': './resampled_bands/T18PVS_20201018T153621_B12_10m_resample.tif',
 '03_10m': './resampled_bands/T18PVS_20201018T153621_B03_10m.jp2',
 '09_10m': './resampled_bands/T18PVS_20201018T153621_B09_10m_resample.tif',
 '11_10m': './resampled_bands/T18PVS_20201018T153621_B11_10m_resample.tif',
 '08_10m': './resampled_bands/T18PVS_20201018T153621_B08_10m.jp2',
 '07_10m': './resampled_bands/T18PVS_20201018T153621_B07_10m_resample.tif',
 '05_10m': './resampled_bands/T18PVS_20201018T153621_B05_10m_resample.tif',
 '06_10m': './resampled_bands/T18PVS_20201018T153621_B06_10m_resample.tif',
 '01_10m': './resampled_bands/T18PVS_20201018T153621_B01_10m_resample.tif',
 '04_10m': './resampled_bands/T18PVS_20201018T153621_B04_10m.jp2',
 '02_10m': './resampled_bands/T18PVS_20201018T153621_B02_10m.jp2'}

## 4.2. Crop bands or raster with a polygon 

### Read the polygon with read_goemtries function

In [105]:
input_json = './polygon_cartagena.json' 

In [106]:
shapes = read_geometries(input_json, 'EPSG:4326', 'EPSG:32618',  file=True)

In [107]:
shapes;

In [108]:
%%time
for shape in shapes:
    # Create a random suffix for every shape
    suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=3))
    for band_key in bands_dpath.keys():
        clip_raster_with_shape(bands_dpath[band_key], [shape], suffix);

...Saving in T18PVS_20201018T153621_B8A_10m_resample_ST9.tif
...Saving in T18PVS_20201018T153621_B12_10m_resample_ST9.tif
...Saving in T18PVS_20201018T153621_B03_10m_ST9.tif
...Saving in T18PVS_20201018T153621_B09_10m_resample_ST9.tif
...Saving in T18PVS_20201018T153621_B11_10m_resample_ST9.tif
...Saving in T18PVS_20201018T153621_B08_10m_ST9.tif
...Saving in T18PVS_20201018T153621_B07_10m_resample_ST9.tif
...Saving in T18PVS_20201018T153621_B05_10m_resample_ST9.tif
...Saving in T18PVS_20201018T153621_B06_10m_resample_ST9.tif
...Saving in T18PVS_20201018T153621_B01_10m_resample_ST9.tif
...Saving in T18PVS_20201018T153621_B04_10m_ST9.tif
...Saving in T18PVS_20201018T153621_B02_10m_ST9.tif
CPU times: user 5.22 s, sys: 224 ms, total: 5.44 s
Wall time: 1.24 s


### 4.3. Open reescaled bands (store their array in a dict) and change the np.dtype

In [109]:
bands_clipped = r'./clipped_bands'

In [110]:
bands_clipped_path = read_bands(bands_clipped, levels=0, suffix='.*')

In [111]:
%%time
arr_bands, reference = instantiate_bands(bands_clipped_path, dtype=np.float64)

CPU times: user 88 ms, sys: 76 ms, total: 164 ms
Wall time: 162 ms


In [112]:
reference;

In [113]:
arr_bands

{'b07': array([[[  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ..., 577., 554.,   0.],
         ...,
         [495., 497., 506., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.]]]),
 'b11': array([[[  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ..., 437., 420.,   0.],
         ...,
         [427., 423., 422., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.]]]),
 'b05': array([[[  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ..., 650., 634.,   0.],
         ...,
         [533., 525., 525., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   0.],
         [  0.,   0.,   0., ...,   0.,   0.,   

## 5. Run model

In [114]:
%%time
out_estimator_path = 'estimator.tif'
estimator_mask = run_estimator(arr_bands, reference, out_estimator_path)

CPU times: user 220 ms, sys: 0 ns, total: 220 ms
Wall time: 220 ms


## 5.1. model to multiband color

In [115]:
estimator_mask.shape

(1, 1788, 989)

In [116]:
# len(estimator_mask[(estimator_mask > 0) & (estimator_mask < 1) ])

In [117]:
cd = cividis(estimator_mask)

In [118]:
cd

array([[[[0.49785974, 0.49785974, 0.49785974, ..., 0.49785974,
          0.49785974, 0.49785974],
         [0.49785974, 0.49785974, 0.49785974, ..., 0.49785974,
          0.49785974, 0.49785974],
         [0.49785974, 0.49785974, 0.49785974, ..., 0.        ,
          0.        , 0.49785974],
         ...,
         [0.        , 0.        , 0.        , ..., 0.49785974,
          0.49785974, 0.49785974],
         [0.49785974, 0.49785974, 0.49785974, ..., 0.49785974,
          0.49785974, 0.49785974],
         [0.49785974, 0.49785974, 0.49785974, ..., 0.49785974,
          0.49785974, 0.49785974]]],


       [[[0.52222119, 0.52222119, 0.52222119, ..., 0.52222119,
          0.52222119, 0.52222119],
         [0.52222119, 0.52222119, 0.52222119, ..., 0.52222119,
          0.52222119, 0.52222119],
         [0.52222119, 0.52222119, 0.52222119, ..., 0.135112  ,
          0.135112  , 0.52222119],
         ...,
         [0.135112  , 0.135112  , 0.135112  , ..., 0.52222119,
          0.52222119, 0

## 5.1 Get the NDWI mask 

In [119]:
%%time
out_path_NDWI = 'NDWI_col.tif'
NDWI_col = NDWI_mask(arr_bands, reference, out_path_NDWI)

CPU times: user 128 ms, sys: 0 ns, total: 128 ms
Wall time: 129 ms


In [120]:
NDWI_col

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])

## 5.2 Composite RGB as the base image

In [121]:
# inpath_rgb = r'./S2A_MSIL2A_20201018T153621_N0214_R068_T18PVS_20201018T193619.SAFE/GRANULE/L2A_T18PVS_A027807_20201018T153841/IMG_DATA/R10m'
# bands = read_bands(inpath, levels=1, suffix='.jp2')
# out_path_rgb = 'rgb_col.tif'
# rgb = composite(inpath_rgb, out_path_rgb, rgb=[4,3,2])


In [122]:
b4 = rio.open(bands_clipped_path['04_10m']).read()
b3 = rio.open(bands_clipped_path['03_10m']).read()
b2 = rio.open(bands_clipped_path['02_10m']).read()
r  = rio.open(bands_clipped_path['01_10m'])

In [123]:
b4

array([[[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ..., 597, 588,   0],
        ...,
        [484, 462, 506, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]]], dtype=uint16)

In [124]:
%%time
emp = rio.open('rgb_clipped.tif','w',driver='Gtiff',
                    width=r.width, 
                    height=r.height, #width and height of any band
                    count=3,
                    crs=r.crs, #coordenate system
                    transform=r.transform, #Transfor from pixel coordinates of source to csr of the input shapes
                    dtype=r.dtypes[0])

#combine the bands RGB in empty raster

emp.write(b2[0],3) #blue
emp.write(b3[0],2) #green
emp.write(b4[0],1) #red
emp.close()

CPU times: user 32 ms, sys: 12 ms, total: 44 ms
Wall time: 44.8 ms


# 6. Masks
## 6.1 Mask model with NDWI

In [125]:
%%time
model = rio.open('estimator.tif').read()
ndwi_mask =  rio.open('NDWI_col.tif').read()
rgb_col = rio.open('rgb_clipped.tif').read()

CPU times: user 88 ms, sys: 8 ms, total: 96 ms
Wall time: 97.3 ms


In [126]:
model.shape

(1, 1788, 989)

In [127]:
%%time
profile = rio.open('estimator.tif').profile
profile.update(dtype=np.float64)
masked_model = np.where(ndwi_mask == 1., 0, model)
with rio.open('masked_model.tif', 'w', **profile) as dst:
    dst.write(masked_model[0], 1)

CPU times: user 96 ms, sys: 0 ns, total: 96 ms
Wall time: 94.2 ms


In [128]:
masked_model.shape

(1, 1788, 989)

## 6.2 Mask RGB with masked_model

### Open original bands from bands dictionary

In [129]:
# b_2 = bands['b02']
# b_3 = bands['b03']
# b_4 = bands['b04']

### Normalize the bands with normalize function and 2.5 factor

In [130]:
b2_n =normalize(b2)*2.5
b3_n =normalize(b3)*2.5
b4_n =normalize(b4)*2.5

In [131]:
print(b4_n.shape)

(1, 1788, 989)


### Masked each band with cividis function and masked model with NDWI

In [132]:
b2_masked = np.where(masked_model == 0, b2_n, cd[0])
b3_masked = np.where(masked_model == 0, b3_n, cd[1])
b4_masked = np.where(masked_model == 0, b4_n, cd[2])

In [133]:
b2_masked.shape

(1, 1788, 989)

### Composite the RGB with new normalize bands (4.3.2)

In [134]:
%%time
masked_rgb = rio.open('rgb_colombia_masked.tif','w',driver='Gtiff',
                        width=r.width, 
                        height=r.height, #width and height of any band
                        count=3,
                        crs=r.crs, #coordenate system
                        transform=r.transform, #Transfor from pixel coordinates of source to csr of the input shapes
                        dtype=np.float64)

    #combine the bands RGB in empty raster
print('Making composite...')
masked_rgb.write(b2_masked[0],3) #blue
masked_rgb.write(b3_masked[0],2) #green
masked_rgb.write(b4_masked[0],1) #red
masked_rgb.close()
print('Done!')

Making composite...
Done!
CPU times: user 56 ms, sys: 64 ms, total: 120 ms
Wall time: 116 ms


In [135]:
inpath = r'./S2A_MSIL2A_20201018T153621_N0214_R068_T18PVS_20201018T193619.SAFE/GRANULE/L2A_T18PVS_A027807_20201018T153841/IMG_DATA/'
bands = read_bands(inpath, levels=1, suffix='.jp2')
outpath = './resampled_bands'
reference_image = bands['02_10m']
# Apply upscale function to all bands with >10m resolution
rescaled_arrays = [upscale(bands[key], outpath,  reference_image) 
                   if key[:3] not in [k[:3] for k in bands.keys() if '10m' in k] 
                   else copy_bands(bands[key], outpath)
                   for key in bands.keys()
                  ] 
bands_dpath = read_bands(outpath, levels=0, suffix='.*')

input_json = './polygon_cartagena.json' 

shapes = read_geometries(input_json, 'EPSG:4326', 'EPSG:32618',  file=True)

for shape in shapes:
    # Create a random suffix for every shape
    suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=3))
    for band_key in bands_dpath.keys():
        clip_raster_with_shape(bands_dpath[band_key], [shape], suffix);
        
        
bands_clipped = r'./clipped_bands'

bands_clipped_path = read_bands(bands_clipped, levels=0, suffix='.*')

arr_bands, reference = instantiate_bands(bands_clipped_path, dtype=np.float64)


out_estimator_path = 'estimator.tif'
estimator_mask = run_estimator(arr_bands, reference, out_estimator_path)


cd = cividis(estimator_mask)

out_path_NDWI = 'NDWI_col.tif'
NDWI_col = NDWI_mask(arr_bands, reference, out_path_NDWI)

b4 = rio.open(bands_clipped_path['04_10m']).read()
b3 = rio.open(bands_clipped_path['03_10m']).read()
b2 = rio.open(bands_clipped_path['02_10m']).read()
r  = rio.open(bands_clipped_path['01_10m'])
profile = r.profile


emp = rio.open('rgb_clipped.tif','w',driver='Gtiff',
                    width=r.width, 
                    height=r.height, #width and height of any band
                    count=3,
                    crs=r.crs, #coordenate system
                    transform=r.transform, #Transfor from pixel coordinates of source to csr of the input shapes
                    dtype=r.dtypes[0])

#combine the bands RGB in empty raster

emp.write(b2[0],3) #blue
emp.write(b3[0],2) #green
emp.write(b4[0],1) #red
emp.close()


model = rio.open('estimator.tif').read()
ndwi_mask =  rio.open('NDWI_col.tif').read()
rgb_col = rio.open('rgb_clipped.tif').read()


profile = rio.open('estimator.tif').profile
profile.update(dtype=np.float64)
masked_model = np.where(ndwi_mask == 1., 0, model)
with rio.open('masked_model.tif', 'w', **profile) as dst:
    dst.write(masked_model[0], 1)

    
b2_n =normalize(b2)*2.5
b3_n =normalize(b3)*2.5
b4_n =normalize(b4)*2.5


b2_masked = np.where(masked_model == 0, b2_n, cd[0])
b3_masked = np.where(masked_model == 0, b3_n, cd[1])
b4_masked = np.where(masked_model == 0, b4_n, cd[2])


masked_rgb = rio.open('rgb_colombia_masked.tif','w',driver='Gtiff',
                        width=r.width, 
                        height=r.height, #width and height of any band
                        count=3,
                        crs=r.crs, #coordenate system
                        transform=r.transform, #Transfor from pixel coordinates of source to csr of the input shapes
                        dtype=np.float64)

    #combine the bands RGB in empty raster
print('Making composite...')
masked_rgb.write(b2_masked[0],3) #blue
masked_rgb.write(b3_masked[0],2) #green
masked_rgb.write(b4_masked[0],1) #red
masked_rgb.close()
print('Done!')

...Saving in T18PVS_20201018T153621_B8A_10m_resample_4XO.tif
...Saving in T18PVS_20201018T153621_B12_10m_resample_4XO.tif
...Saving in T18PVS_20201018T153621_B03_10m_4XO.tif
...Saving in T18PVS_20201018T153621_B09_10m_resample_4XO.tif
...Saving in T18PVS_20201018T153621_B11_10m_resample_4XO.tif
...Saving in T18PVS_20201018T153621_B08_10m_4XO.tif
...Saving in T18PVS_20201018T153621_B07_10m_resample_4XO.tif
...Saving in T18PVS_20201018T153621_B05_10m_resample_4XO.tif
...Saving in T18PVS_20201018T153621_B06_10m_resample_4XO.tif
...Saving in T18PVS_20201018T153621_B01_10m_resample_4XO.tif
...Saving in T18PVS_20201018T153621_B04_10m_4XO.tif
...Saving in T18PVS_20201018T153621_B02_10m_4XO.tif
Making composite...
Done!


In [139]:
x = rio.open('masked_model.tif').profile
x

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 989, 'height': 1788, 'count': 1, 'crs': CRS.from_epsg(32618), 'transform': Affine(10.0, 0.0, 433780.0,
       0.0, -10.0, 1140370.0), 'blockxsize': 1024, 'blockysize': 1024, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}

In [142]:
x.update(crs='CRS.from_epsg(4326)')
x

{'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 989, 'height': 1788, 'count': 1, 'crs': 'CRS.from_epsg(4326)', 'transform': Affine(10.0, 0.0, 433780.0,
       0.0, -10.0, 1140370.0), 'blockxsize': 1024, 'blockysize': 1024, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}

In [143]:
outpath = 'plastic_shape.shp'
shapes, coordenates = get_coords(x, outpath)

TypeError: invalid path or file: {'driver': 'GTiff', 'dtype': 'float64', 'nodata': None, 'width': 989, 'height': 1788, 'count': 1, 'crs': 'CRS.from_epsg(4326)', 'transform': Affine(10.0, 0.0, 433780.0,
       0.0, -10.0, 1140370.0), 'blockxsize': 1024, 'blockysize': 1024, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}

In [ ]:
rio.open('masked_model.tif').profile

In [156]:
with rio.open('masked_model.tif') as src:
    profile = src.profile.update(crs='CRS.from_epsg(4326)')
    file = src.read(1).astype(np.float32)
    coordenates = []
    coordenate_list = []

for shp, val in rio.features.shapes(file, transform=profile['transform']):
    coordenates.append(shape(shp))
    coordenate_list.append(shp)

d = {'geometry': coordenates}
gdf = gpd.GeoDataFrame(d, index=range(len(coordenates))).set_crs(epsg=32618)
gdf.proyect()
gdf.to_file(driver='ESRI Shapefile', filename=outpath) 
 

    

TypeError: 'NoneType' object is not subscriptable

In [153]:
file

array([[0.4999912, 0.4999912, 0.4999912, ..., 0.4999912, 0.4999912,
        0.4999912],
       [0.4999912, 0.4999912, 0.4999912, ..., 0.4999912, 0.4999912,
        0.4999912],
       [0.4999912, 0.4999912, 0.4999912, ..., 0.       , 0.       ,
        0.4999912],
       ...,
       [0.       , 0.       , 0.       , ..., 0.4999912, 0.4999912,
        0.4999912],
       [0.4999912, 0.4999912, 0.4999912, ..., 0.4999912, 0.4999912,
        0.4999912],
       [0.4999912, 0.4999912, 0.4999912, ..., 0.4999912, 0.4999912,
        0.4999912]], dtype=float32)

In [17]:
gdf.project?

In [ ]:
your_gdf.crs = "EPSG:4326"

In [ ]:
    coordenates = []
    coordenate_list = []
    for shp, val in rio.features.shapes(file, transform=profile['transform']):
        coordenates.append(shape(shp))
        coordenate_list.append(shp)

d = {'geometry': coordenates}
gdf = gpd.GeoDataFrame(d, index=range(len(coordenates))).to_crs('EPSG:32631')
gdf.to_file(driver='ESRI Shapefile', filename=outpath)  


return gdf, coordenate_list

In [ ]:
shapes